<a href="https://colab.research.google.com/github/grackner/dsc_261_final_project/blob/main/qwen_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Qwen Text Generation

url: https://huggingface.co/Qwen/Qwen2-VL-2B-Instruct

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import uuid
import pandas as pd
import kagglehub
from transformers import AutoProcessor, AutoModelForVision2Seq

In [ ]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")
model = AutoModelForVision2Seq.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

In [ ]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Qwen2VLForConditionalGeneration(
  (model): Qwen2VLModel(
    (visual): Qwen2VisionTransformerPretrainedModel(
      (patch_embed): PatchEmbed(
        (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
      )
      (rotary_pos_emb): VisionRotaryEmbedding()
      (blocks): ModuleList(
        (0-31): 32 x Qwen2VLVisionBlock(
          (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
          (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
          (attn): VisionAttention(
            (qkv): Linear(in_features=1280, out_features=3840, bias=True)
            (proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (mlp): VisionMlp(
            (fc1): Linear(in_features=1280, out_features=5120, bias=True)
            (act): QuickGELUActivation()
            (fc2): Linear(in_features=5120, out_features=1280, bias=True)
          )
        )
      )
      (merger): PatchMerger(
        (ln_q): LayerN

In [ ]:
# Test inference
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "who is michael jordan?"}
        ]
    },
]
inputs = processor.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(processor.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Michael Jordan is an American former professional basketball player, widely regarded as one of the greatest basketball players of all time. He played for the Chicago Bulls in the National Basketball Association (NBA) from 


In [ ]:
# Create test dataset
qwen_df = pd.DataFrame(columns=['uuid', 'generated_article'])

In [ ]:
def generate_article():
  messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": f"Write the body of a news article, do not include a headline or title. It can be about any topic"}
        ]
    },
  ]
  inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
  ).to(model.device)

  outputs = model.generate(**inputs, max_new_tokens=100, do_sample=True, temperature=0.9, top_p=0.95,top_k=50)
  response = processor.decode(outputs[0][inputs["input_ids"].shape[-1]:])
  return response

In [ ]:
# Generate articles
n = 100 # Number of datapoints to create
for i in range(0, n + 1):
  # Add unique identifier for the row
  qwen_df.loc[i, 'uuid'] = str(uuid.uuid4())
  response = generate_article()
  qwen_df.loc[i, 'generated_article'] = response

In [ ]:
qwen_df.head()

,uuid,generated_article
0,76cf2ce8-d643-4882-9c66-b0ee34921f30,On the heels of an exciting announcement made ...
1,91383ee7-7098-4b30-9837-078d80f4a22d,"After months of speculation and anticipation, ..."
2,d818c8fc-f95d-45d4-a0ea-b432322c1f75,"On Monday, authorities in the United States re..."
3,c17bec88-4c1e-44ce-a1b3-baf055d20c11,"""Local Business Leaders Call for Increased Edu..."
4,54755121-78f8-41f9-a5fc-8327a31b3fec,"""Local Business Grows with New Headquarters""<|..."


In [ ]:
qwen_df.to_csv("qwen_outputs.csv")

In [ ]:
## Load dataset
# path = kagglehub.dataset_download("gowrishankarp/newspaper-text-summarization-cnn-dailymail")

# print("Path to dataset files:", path)

# train_df = pd.read_csv(path + "/cnn_dailymail/train.csv")
# test_df = pd.read_csv(path + "/cnn_dailymail/test.csv")
# val_df = pd.read_csv(path + "/cnn_dailymail/validation.csv")

Using Colab cache for faster access to the 'newspaper-text-summarization-cnn-dailymail' dataset.
Path to dataset files: /kaggle/input/newspaper-text-summarization-cnn-dailymail


In [ ]:
## Loop through the train dataset
# for index, row in train_df.head(1).iterrows():
#   print(index)
#   # Get article
#   article = row["article"]
#   # Cut down to 100 words
#   article = " ".join(article.split()[:100])
#   # Format the query
#   messages = [
#     {
#         "role": "user",
#         "content": [
#             {"type": "text", "text": f"Write a news article do not include a title"}
#             # {"type": "text", "text": f"Write an article with a similar style to the following article example from CNN: {article}"}
#         ]
#     },
#   ]
#   # Store query in df # TODO: Should happen in data cleaning?
#   print(article)
#   train_df.loc[index, 'query_article'] = messages[0]['content'][0]['text']
#   ## Run query through inference
#   inputs = processor.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     tokenize=True,
#     return_dict=True,
#     return_tensors="pt",
#   ).to(model.device)

#   outputs = model.generate(**inputs, max_new_tokens=100)
#   response = processor.decode(outputs[0][inputs["input_ids"].shape[-1]:])
#   # Store answer in df
#   train_df.loc[index, 'model_output'] = response

0
By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and
